In [ ]:
def normal_at(shape, world_point):
    """
    >>> s = sphere()
    >>> n = normal_at(s, point(1,0,0))
    >>> n == vector(1,0,0)
    array([ True,  True,  True,  True])

    >>> s = sphere()
    >>> n = normal_at(s, point(0,1,0))
    >>> n == vector(0,1,0)
    array([ True,  True,  True,  True])

    >>> s = sphere()
    >>> n = normal_at(s, point(0,0,1))
    >>> n == vector(0,0,1)
    array([ True,  True,  True,  True])

    >>> s = sphere()
    >>> n = normal_at(s, point(np.sqrt(3)/3,np.sqrt(3)/3,np.sqrt(3)/3))
    >>> n == vector(np.sqrt(3)/3,np.sqrt(3)/3,np.sqrt(3)/3)
    array([ True,  True,  True,  True])

    >>> s = sphere()
    >>> n = normal_at(s, point(np.sqrt(3)/3,np.sqrt(3)/3,np.sqrt(3)/3))
    >>> n == normalize(n)
    array([ True,  True,  True,  True])

    >>> s = sphere()
    >>> s.transform = translation(0,1,0)
    >>> n = normal_at(s, point(0, 1.70711, -0.70711))
    >>> np.isclose(n,vector(0, 0.70711, -0.70711))
    array([ True,  True,  True,  True])

    >>> s = sphere()
    >>> m = matrix_multiply(scaling(1,0.5,1), rotation_z(np.pi / 5))
    >>> s.transform = m
    >>> n = normal_at(s, point(0, np.sqrt(2)/2, -np.sqrt(2)/2))
    >>> np.isclose(n,vector(0, 0.97014, -0.242535625))
    array([ True,  True,  True,  True])
    """
    return shape.normal_at(world_point)

def reflect(inp, norm):
    """
    >>> v = vector(1,-1,0)
    >>> n = vector(0,1,0)
    >>> r = reflect(v,n)
    >>> r == vector(1,1,0)
    array([ True,  True,  True,  True])

    >>> v = vector(0,-1,0)
    >>> n = vector(np.sqrt(2)/2, np.sqrt(2)/2, 0)
    >>> r = reflect(v,n)
    >>> np.isclose(r,vector(1,0,0))
    array([ True,  True,  True,  True])
    """
    return inp - norm * 2 * dot(inp, norm)

class Light(object):
    def __init__(self):
        pass

class PointLight(Light):
    def __init__(self, position, intensity):
        self.position = position
        self.intensity = intensity

def point_light(position, intensity):
    """
    >>> i = color(1,1,1)
    >>> p = point(0,0,0)
    >>> light = point_light(p,i)
    >>> light.position == p
    array([ True,  True,  True,  True])

    >>> light.intensity == i
    array([ True,  True,  True])

    """
    return PointLight(position, intensity)

class Material(object):
    def __init__(self, color, ambient, diffuse, specular, shininess, reflective=0.0, transparency=0.0, refractive_index=1.0):
        if ambient < 0 or diffuse < 0 or specular < 0 or shininess < 0:
            raise ValueError("Materials expect non-negative floating point values.")
        self.color = color
        self.ambient = np.float64(ambient)
        self.diffuse = np.float64(diffuse)
        self.specular = np.float64(specular)
        self.shininess = np.float64(shininess)
        self.pattern = None
        self.reflective = reflective
        self.transparency = transparency
        self.refractive_index = refractive_index

    def __repr__(self):
        return "c: {} a: {} d: {} sp: {} sh: {}".format(self.color, self.ambient, self.diffuse, self.specular, self.shininess)

def material():
    """
    >>> m = material()
    >>> m.color == color(1,1,1)
    array([ True,  True,  True])

    >>> m.ambient == 0.1 and m.diffuse == 0.9 and m.specular == 0.9 and m.shininess == 200.0
    True

    >>> s = sphere()
    >>> sm = s.material
    >>> m = material()
    >>> sm.color == m.color
    array([ True,  True,  True])
    >>> sm.ambient == m.ambient and sm.diffuse == m.diffuse and sm.specular == m.specular and sm.shininess == m.shininess
    True

    >>> s = sphere()
    >>> m = material()
    >>> m.ambient = 1
    >>> s.material = m
    >>> s.material.ambient == 1
    True

    >>> m = material()
    >>> m.reflective == 0
    True

    >>> m.transparency == 0
    True
    >>> m.refractive_index == 1
    True
    """
    return Material(color(1,1,1),0.1,0.9,0.9,200.0)

black = color(0,0,0)
def lighting(material, shape, light, point, eyev, normalv, in_shadow=False):
    """
    >>> m = material()
    >>> shape = sphere()
    >>> pos = point(0,0,0)
    >>> eyev = vector(0,0,-1)
    >>> normalv = vector(0,0,-1)
    >>> light = point_light(point(0,0,-10), color(1,1,1))
    >>> result = lighting(m, shape, light, pos, eyev, normalv)
    >>> np.isclose(result, color(1.9,1.9,1.9))
    array([ True,  True,  True])

    >>> m = material()
    >>> pos = point(0,0,0)
    >>> eyev = vector(0,np.sqrt(2)/2,-np.sqrt(2)/2)
    >>> normalv = vector(0,0,-1)
    >>> light = point_light(point(0,0,-10), color(1,1,1))
    >>> result = lighting(m, shape, light, pos, eyev, normalv)
    >>> np.isclose(result, color(1.0,1.0,1.0))
    array([ True,  True,  True])

    >>> m = material()
    >>> pos = point(0,0,0)
    >>> eyev = vector(0,0,-1)
    >>> normalv = vector(0,0,-1)
    >>> light = point_light(point(0,10,-10), color(1,1,1))
    >>> result = lighting(m, shape, light, pos, eyev, normalv)
    >>> np.isclose(result, color(0.7364, 0.7364, 0.7364))
    array([ True,  True,  True])

    >>> m = material()
    >>> pos = point(0,0,0)
    >>> eyev = vector(0,-np.sqrt(2)/2,-np.sqrt(2)/2)
    >>> normalv = vector(0,0,-1)
    >>> light = point_light(point(0,10,-10), color(1,1,1))
    >>> result = lighting(m, shape, light, pos, eyev, normalv)
    >>> np.isclose(result, color(1.6364, 1.6364, 1.6364))
    array([ True,  True,  True])

    >>> m = material()
    >>> pos = point(0,0,0)
    >>> eyev = vector(0,0,-1)
    >>> normalv = vector(0,0,-1)
    >>> light = point_light(point(0,0,10), color(1,1,1))
    >>> result = lighting(m, shape, light, pos, eyev, normalv)
    >>> np.isclose(result, color(0.1, 0.1, 0.1))
    array([ True,  True,  True])

    >>> m = material()
    >>> pos = point(0,0,0)
    >>> eyev = vector(0,0,-1)
    >>> normalv = vector(0,0,-1)
    >>> light = point_light(point(0,0,-10), color(1,1,1))
    >>> in_shadow = True
    >>> result = lighting(m, shape, light, pos, eyev, normalv, in_shadow)
    >>> np.isclose(result, color(0.1, 0.1, 0.1))
    array([ True,  True,  True])

    >>> m = material()
    >>> shape = sphere()
    >>> eyev = vector(0,0,-1)
    >>> normalv = vector(0,0,-1)
    >>> light = point_light(point(0,0,-10), color(1,1,1))
    >>> in_shadow = False
    >>> m.pattern = stripe_pattern(color(1,1,1), color(0,0,0))
    >>> m.ambient = 1
    >>> m.diffuse = 0
    >>> m.specular = 0
    >>> c1 = lighting(m, shape, light, point(0.9,0,0), eyev, normalv, in_shadow)
    >>> c2 = lighting(m, shape, light, point(1.1,0,0), eyev, normalv, in_shadow)
    >>> np.isclose(c1, color(1,1,1))
    array([ True,  True,  True])

    >>> np.isclose(c2, color(0,0,0))
    array([ True,  True,  True])
    """
    if material.pattern is not None:
        pcolor = material.pattern.pattern_at_shape(shape, point)
    else:
        pcolor = material.color

    effective_color = pcolor * light.intensity
    ambient = effective_color * material.ambient
    if in_shadow:
        return ambient

    lightv = normalize(light.position - point)
    light_dot_normal = dot(lightv, normalv)
    if light_dot_normal < 0:
        diffuse = black
        specular = black
    else:
        diffuse = effective_color * material.diffuse * light_dot_normal
        reflectv = reflect(-lightv, normalv)
        reflect_dot_eye = dot(reflectv, eyev)
        if reflect_dot_eye <= 0:
            specular = black
        else:
            factor = np.power(reflect_dot_eye, material.shininess)
            specular = light.intensity * material.specular * factor
    return ambient + diffuse + specular